In [2]:
from transformers import TextClassificationPipeline, DistilBertTokenizer, DistilBertForSequenceClassification,AutoTokenizer, RobertaTokenizer, RobertaForSequenceClassification, BertForSequenceClassification, BertTokenizer, AlbertForSequenceClassification, AlbertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy
import torch

c:\Users\Matin\Desktop\Examensarbete 2025\WPFN\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('datasets/fake_or_real_news_cleaned.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    6335 non-null   object
 1   label   6335 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 99.1+ KB


In [5]:
roberta_path = r'models\fake-news-roberta'
bert_path = r'models\fake-news-bert'
albert_path = r'models\fake-news-albert'

In [6]:
roberta_model = RobertaForSequenceClassification.from_pretrained(roberta_path, num_labels=2)
bert_model = BertForSequenceClassification.from_pretrained(bert_path, num_labels=2)
albert_model = AlbertForSequenceClassification.from_pretrained(albert_path, num_labels=2)

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
albert_tokenizer = AutoTokenizer.from_pretrained('albert-base-v2')

In [7]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
texts = test_df['text'].tolist()
true_labels = test_df['label'].tolist()

In [9]:
texts = [text[:512] for text in texts]

In [37]:
batch_size = 2
predicted_labels = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    inputs = roberta_tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
    outputs = roberta_model(**inputs)

    logits = outputs.logits  # [batch_size, num_labels]
    batch_preds = torch.argmax(logits, dim=1)  # [batch_size]
    predicted_labels.extend(batch_preds.tolist())  # [0, 1, 2, ...]



In [51]:
y_true = true_labels
y_pred = predicted_labels

In [52]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       639
           1       1.00      1.00      1.00       628

    accuracy                           1.00      1267
   macro avg       1.00      1.00      1.00      1267
weighted avg       1.00      1.00      1.00      1267



In [54]:
batch_size = 2
predicted_labels = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    inputs = bert_tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
    outputs = bert_model(**inputs)

    logits = outputs.logits  # [batch_size, num_labels]
    batch_preds = torch.argmax(logits, dim=1)  # [batch_size]
    predicted_labels.extend(batch_preds.tolist()) # [0, 1, 0, 1]

In [55]:
y_true = true_labels
y_pred = predicted_labels
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       639
           1       0.99      0.99      0.99       628

    accuracy                           0.99      1267
   macro avg       0.99      0.99      0.99      1267
weighted avg       0.99      0.99      0.99      1267



In [10]:
batch_size = 2
predicted_labels = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    inputs = albert_tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
    outputs = albert_model(**inputs)

    logits = outputs.logits  # [batch_size, num_labels]
    batch_preds = torch.argmax(logits, dim=1)  # [batch_size]
    predicted_labels.extend(batch_preds.tolist()) # [0, 1, 0, 1]

In [11]:
y_true = true_labels
y_pred = predicted_labels
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       639
           1       0.98      1.00      0.99       628

    accuracy                           0.99      1267
   macro avg       0.99      0.99      0.99      1267
weighted avg       0.99      0.99      0.99      1267

